In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
data = pd.read_csv("phishing.csv")
data.head(5)

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


In [8]:
data = data.drop("Index",axis=1)
for column in data.columns:
    print(data[column].unique())

[ 1 -1]
[ 1  0 -1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[-1  1]
[ 0 -1  1]
[ 1 -1  0]
[-1  1]
[ 1 -1]
[ 1 -1]
[-1  1]
[ 1 -1]
[ 0 -1  1]
[-1  0  1]
[-1  1  0]
[ 1 -1]
[ 1 -1]
[0 1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[-1  1]
[-1  1]
[ 0  1 -1]
[-1  1]
[ 1 -1]
[ 1  0 -1]
[ 1 -1]
[-1  1]


In [9]:
X = data.drop("class",axis=1)
Y = data["class"]
Y = Y.map({-1:0,1:1})

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=0.7) 
X_train.shape

(7737, 30)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(30,activation = 'relu'),
    Dense(20,activation = 'relu'),
    Dense(10,activation = 'relu'),
    Dense(1,activation = 'sigmoid'),
])
model.compile(optimizer="adam",loss=BinaryCrossentropy())
model.fit(X_train,Y_train,epochs=100)

Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3424
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1924
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1746
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1646
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 0.1549
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1454
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1350
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.1286
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1236
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1164
Epoch 11/100
242/242 [==============================] - 1s 2ms/step - loss: 0.1122
Epoch 12/100
242/242 [==============================] - 0s 2ms/step - loss: 0.1089
Epoch 13/100


In [13]:
from sklearn.metrics import accuracy_score
predictions = model.predict(X_train)

# Convert predictions to binary values (-1 or 1) based on a threshold (0.5 for sigmoid activation)
predictions_binary = (predictions > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(Y_train, predictions_binary)

print("Test Accuracy:", accuracy)
print(predictions)

242/242 [==============================] - 0s 1ms/step
Test Accuracy: 0.983714618069019
[[9.9999690e-01]
 [9.9004787e-01]
 [8.1669592e-32]
 ...
 [5.8213220e-05]
 [9.9999392e-01]
 [1.3878708e-14]]


In [14]:
model.save("phishing-website-detector.h5")